In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input/kernel-methods-ammi-2023/Xtr0.csv'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch

In [ ]:
Xtr0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr0.csv")
Ytr0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr0.csv")

Xtr1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr1.csv")
Ytr1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr1.csv")

Xtr2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xtr2.csv")
Ytr2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Ytr2.csv")

In [ ]:
Xte0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte0.csv")
# Yte0=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Yte0.csv")

Xte1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte1.csv")
# Yte1=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Yte1.csv")

Xte2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Xte2.csv")
# Yte2=pd.read_csv("/kaggle/input/kernel-methods-ammi-2023/Yte2.csv")

In [ ]:
Ytr0

In [ ]:
Ytr0.Bound.hist()

In [ ]:
Ytr1.Bound.hist()

In [ ]:
Ytr2.Bound.hist()

In [ ]:
dataTr0=pd.concat([Xtr0, Ytr0[["Bound"]]], axis=1, join='inner')
dataTr1=pd.concat([Xtr1, Ytr1[["Bound"]]], axis=1, join='inner')
dataTr2=pd.concat([Xtr2, Ytr2[["Bound"]]], axis=1)#, join='inner')




In [ ]:

def one_hot_encode(seq):
    """
    Given a DNA sequence, return its one-hot encoding
    """
    # Make sure seq has only allowed bases
    allowed = set("ACTGN")
    if not set(seq).issubset(allowed):
        invalid = set(seq) - allowed
        raise ValueError(f"Sequence contains chars not in allowed DNA alphabet (ACGTN): {invalid}")

    # Dictionary returning one-hot encoding for each nucleotide
    nuc_d = {'A':[1.0,0.0,0.0,0.0],
             'C':[0.0,1.0,0.0,0.0],
             'G':[0.0,0.0,1.0,0.0],
             'T':[0.0,0.0,0.0,1.0],
             'N':[0.0,0.0,0.0,0.0]}

    # Create array from nucleotide sequence
    vec=np.array([nuc_d[x] for x in seq])

    return vec

In [ ]:
def OneHot(df,seq_col='seq',target_col=None):
        # +--------------------+
        # | Get the X examples |
        # +--------------------+
        # extract the DNA from the appropriate column in the df
        seqs = list(df[seq_col].values)
        seq_len = len(seqs[0])

        # one-hot encode sequences, then stack in a torch tensor
         #self.ohe_seqs = torch.stack([torch.tensor(one_hot_encode(x) for x in self.seqs])
        ohe_seqs = np.stack([one_hot_encode(x).flatten() for x in seqs])
        n,p= ohe_seqs.shape
        #         self.ohe_seqs=self.add_ones(self.ohe_seqs)
        print(n,p)
        if target_col is not None:
            labels = (df[target_col].values)
            print( labels.shape)
            return ohe_seqs ,labels
        return ohe_seqs


In [ ]:
x_train_0,y_train_0 = OneHot(dataTr0,target_col="Bound")

x_train_1,y_train_1 = OneHot(dataTr1,target_col="Bound")

x_train_2,y_train_2 = OneHot(dataTr2,target_col="Bound")

Xte0=OneHot(Xte0)
Xte1=OneHot(Xte1)
Xte2=OneHot(Xte2)

In [ ]:
class LogisticRegression:
    '''
    The goal of this class is to create a LogisticRegression class,
    that we will use as our model to classify data point into a corresponding class
    '''
    def __init__(self,lam,lr,n_epochs,eps=1e-12):
        self.lr = lr
        self.lam =lam
        self.eps=eps
        self.n_epochs = n_epochs
        self.train_losses = []
        self.w = None
        self.weight = []
    def add_ones(self, x):
        ##### WRITE YOUR CODE HERE #####
        return np.hstack((np.ones((x.shape[0],1)),x))
        #pass
        #### END CODE ####

    def sigmoid(self, x):
        ##### WRITE YOUR CODE HERE ####
        return 1/(1+np.exp(-x@self.w))
        #pass
        #### END CODE ####

    def cross_entropy(self, x, y_true):
        ##### WRITE YOUR CODE HERE #####
        y_pred = self.sigmoid(x)
        loss =-np.sum( y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))/x.shape[0]
        return loss
        #### END CODE ####

    def predict_proba(self,x):  #This function will use the sigmoid function to compute the probalities
        ##### WRITE YOUR CODE HERE #####
        #x= self.add_ones(x)
        proba = self.sigmoid(x)
        return proba
        #### END CODE ####

    def predict(self,x):
        ##### WRITE YOUR CODE HERE #####
        probas = self.predict_proba(self.add_ones(x))
        output=np.zeros(probas.shape[0])
        for i in range(probas.shape[0]):
            if probas[i]>=0.5: #convert the probalities into 0 and 1 by using a treshold=0.5
                output[i]=1
            else:
                output[i]=0


        return output
      #### END CODE ####

    def fit(self,x,y):
        # Add ones to x
        x=self.add_ones(x)


        # reshape y if needed
        if y.ndim==1:
             y=y.reshape(-1,1)

        # Initialize w to zeros vector >>> (x.shape[1])
        self.w=np.zeros((x.shape[1],1))

        for epoch in range(self.n_epochs):

              # make predictions
            self.ypred = self.sigmoid(x)

              #compute the gradient
            dl = (-1/y.shape[1])*x.T@(y-self.ypred.reshape(-1,1))+2*self.lam*self.w
#             if np.mean((self.lam*dl)**2)<self.eps:
#                 break
              #update rule
            self.w=self.w- self.lr*dl

              #Compute and append the training loss in a list
            loss = self.cross_entropy(x, y)
            #print(x.shape)
            self.train_losses.append(loss)

            if epoch%100 == 0:
                print(f'loss for epoch {epoch}  : {loss}')


    def accuracy(self,y_true, y_pred):
        ##### WRITE YOUR CODE HERE #####
        acc = 0
        for i in range(y_true.shape[0]):
            if y_true[i]==y_pred[i]:
                acc+=1
        return (acc/y_true.shape[0])*100
        #### END CODE ####

In [ ]:
print("-----------Training Xtr0-------------")
model = LogisticRegression(lam=0.1,lr=0.0001,n_epochs=20000,eps=1e-12)
model.fit(x_train_0,y_train_0)
Yte0=model.predict(Xte0)
print("-----------Training Xtr1-------------")
#model.accuracy(y_test, y_pred)
model = LogisticRegression(lam=0.1,lr=0.0001,n_epochs=20000,eps=1e-12)
model.fit(x_train_1,y_train_1)
Yte1=model.predict(Xte1)
print("-----------Training Xtr2-------------")
model = LogisticRegression(lam=0.1,lr=0.0001,n_epochs=20000,eps=1e-12)
model.fit(x_train_2,y_train_2)
Yte2=model.predict(Xte2)





In [ ]:
Yte0=np.asarray(Yte0, dtype = 'int')
Yte1=np.asarray(Yte1, dtype = 'int')
Yte2=np.asarray(Yte2, dtype = 'int')
Yte=np.concatenate(((Yte0,Yte1,Yte2)))
Id=np.arange(len(Yte))
Yte = pd.DataFrame(zip(Id,Yte),columns=["Id","Bound"]).set_index('Id')

In [ ]:
Yte.to_csv("Yte.csv")